## Parte 0: Gerando o Cenário (O Problema)
Vamos gerar um arquivo CSV com 1 a 5 milhões de linhas para simular o "peso" de carregar dados.

In [1]:
import pandas as pd
import numpy as np
import time
import os

# Garantir que a pasta data existe
os.makedirs('../data', exist_ok=True)

# Gerando dados dummy de Vendas (simulando a tabela do slide 4)
print("Gerando CSV de vendas... (Isso pode levar alguns segundos)")
df = pd.DataFrame({
    'id_pedido': range(1, 1000001),
    'data': pd.date_range(start='2023-01-01', periods=1000000, freq='s'),
    'regiao': np.random.choice(['Norte', 'Sul', 'Leste', 'Oeste'], 1000000),
    'produto': np.random.choice(['Mouse', 'Teclado', 'Monitor', 'Cabo'], 1000000),
    'valor': np.random.uniform(10, 500, 1000000).round(2)
})
df.to_csv('../data/vendas_big.csv', index=False)
print("CSV gerado com sucesso em ../data/vendas_big.csv!")

Gerando CSV de vendas... (Isso pode levar alguns segundos)
CSV gerado com sucesso em ../data/vendas_big.csv!


## Parte 1: O "Canhão" (Abordagem Pandas Tradicional)
Aqui demonstramos a abordagem criticada, onde carregamos o arquivo inteiro para a memória RAM só para fazer uma agregação simples.

In [2]:
# O jeito "Sandbox Pesado"
start = time.time()

# 1. Carrega tudo para memória (Gargalo de I/O e RAM)
print("Carregando CSV com Pandas...")
df_pandas = pd.read_csv('../data/vendas_big.csv')

# 2. Processa lógica pythonica
resultado = df_pandas.groupby('regiao')['valor'].sum()

end = time.time()
print(f"Tempo Pandas: {end - start:.4f} segundos")
print(resultado)

Carregando CSV com Pandas...
Tempo Pandas: 0.5987 segundos
regiao
Leste    63937663.84
Norte    63728638.70
Oeste    63685725.54
Sul      63820057.05
Name: valor, dtype: float64


> **Nota:** Observe o tempo gasto. Se o arquivo tivesse 10GB, sua máquina provavelmente travaria ou o processo seria morto por falta de memória (OOM).

## Parte 2: A "Ferramenta Certa" (DuckDB em CSV)
Agora usamos o DuckDB lendo o CSV diretamente. Ele é um motor OLAP *in-process* que consegue executar SQL em arquivos sem carregá-los totalmente na RAM.

In [4]:
import duckdb

# O jeito "DuckDB" - SQL direto no arquivo
start = time.time()

# Note que usamos o arquivo CSV como se fosse uma tabela
query = """
SELECT 
    regiao, 
    SUM(valor) as total_vendas
FROM '../data/vendas_big.csv'
GROUP BY regiao
"""
duckdb.sql(query).show()

end = time.time()
print(f"Tempo DuckDB (CSV direto): {end - start:.4f} segundos")

┌─────────┬────────────────────┐
│ regiao  │    total_vendas    │
│ varchar │       double       │
├─────────┼────────────────────┤
│ Sul     │  63820057.05000006 │
│ Oeste   │ 63685725.539999776 │
│ Norte   │   63728638.7000001 │
│ Leste   │  63937663.83999974 │
└─────────┴────────────────────┘

Tempo DuckDB (CSV direto): 0.1300 segundos


## Parte 3: Otimização da Arquitetura (Parquet)
Para atingir a performance ideal para Agentes de IA (que precisam de respostas em sub-segundos), convertemos os dados para **Parquet** (formato colunar).

In [5]:
# 1. Converter CSV para Parquet (Simulando a etapa de Ingestão)
print("Convertendo para Parquet...")
duckdb.sql("COPY (SELECT * FROM '../data/vendas_big.csv') TO '../data/vendas.parquet' (FORMAT PARQUET)")

# 2. Consultar no Parquet
start = time.time()
duckdb.sql("SELECT regiao, SUM(valor) FROM '../data/vendas.parquet' GROUP BY regiao").show()
end = time.time()

print(f"Tempo DuckDB (Parquet): {end - start:.4f} segundos")

Convertendo para Parquet...
┌─────────┬────────────────────┐
│ regiao  │     sum(valor)     │
│ varchar │       double       │
├─────────┼────────────────────┤
│ Leste   │ 63937663.839999765 │
│ Norte   │  63728638.70000008 │
│ Oeste   │  63685725.53999995 │
│ Sul     │  63820057.05000003 │
└─────────┴────────────────────┘

Tempo DuckDB (Parquet): 0.0045 segundos


## Parte 4: Materializando o Banco e Simulando IA
Muitos acham que a IA faz mágica. Aqui vamos mostrar a engenharia por trás: **Function Calling**.

Primeiro, vamos criar um arquivo de banco de dados persistente (`.duckdb`).

In [6]:
print("Materializando banco de dados DuckDB persistente...")

# Conecta (ou cria) o arquivo físico
con = duckdb.connect('../data/vendas.duckdb')

# Cria a tabela 'vendas' dentro do arquivo .duckdb a partir do Parquet
con.sql("""
    CREATE OR REPLACE TABLE vendas AS 
    SELECT * FROM '../data/vendas.parquet'
""")

print("Banco '../data/vendas.duckdb' criado com sucesso!")
con.sql("DESCRIBE vendas").show()

con.close()

Materializando banco de dados DuckDB persistente...
Banco '../data/vendas.duckdb' criado com sucesso!
┌─────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name │ column_type │  null   │   key   │ default │  extra  │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ id_pedido   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ data        │ TIMESTAMP   │ YES     │ NULL    │ NULL    │ NULL    │
│ regiao      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ produto     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ valor       │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘



### O "Truque" da Mágica: Simulando Function Calling
Vamos simular o fluxo: `Usuário -> LLM (Mock) -> SQL -> DuckDB`.
A IA não executa o código, ela apenas gera o SQL (Function Call) que nosso motor determinístico executa.

In [7]:
import json

# ---------------------------------------------------------
# 1. O "Prompt de Sistema" (O que enviamos para a IA)
# ---------------------------------------------------------
schema_info = """
Tabela: vendas
Colunas: 
- data (TIMESTAMP)
- regiao (STRING): 'Norte', 'Sul', 'Leste', 'Oeste'
- produto (STRING)
- valor (FLOAT)
"""

print(f"--- Contexto do Sistema ---\n{schema_info}\n")


# ---------------------------------------------------------
# 2. A "Mágica" Simulada (Mock do LLM)
# ---------------------------------------------------------
def mock_llm_function_call(user_question):
    """
    Simula um LLM recebendo uma pergunta e decidindo qual SQL gerar.
    Na vida real, aqui seria uma chamada para openai.chat.completions.create
    """
    print(f"🤖 AI Recebeu: '{user_question}'")
    print("🤖 AI Pensando... (Gerando SQL baseado no Schema)")
    
    # MOCK: Vamos fingir que a IA gerou isso deterministicamente
    if "total" in user_question.lower() and "norte" in user_question.lower():
        # A IA retorna um JSON estruturado, não apenas texto solto
        return {
            "function_name": "executar_sql_analitico",
            "arguments": {
                "sql_query": "SELECT SUM(valor) as total FROM vendas WHERE regiao = 'Norte'",
                "explicacao": "Calculando a soma da coluna valor filtrando pela região Norte."
            }
        }
    elif "top" in user_question.lower() and "produtos" in user_question.lower():
        return {
            "function_name": "executar_sql_analitico",
            "arguments": {
                "sql_query": "SELECT produto, COUNT(*) as qtd FROM vendas GROUP BY produto ORDER BY qtd DESC LIMIT 3",
                "explicacao": "Agrupando por produto e ordenando por contagem decrescente."
            }
        }
    else:
        return None

# ---------------------------------------------------------
# 3. A Função Determinística (O Canhão Silenciado)
# ---------------------------------------------------------
def executar_sql_analitico(sql_query):
    """
    Esta é a ferramenta que a IA decidiu usar.
    Ela roda em ambiente controlado (DuckDB), isolado e seguro.
    """
    print(f"⚙️  Executando no DuckDB: {sql_query}")
    
    # Conecta no banco persistente que criamos
    con = duckdb.connect('../data/vendas.duckdb')
    
    try:
        df_resultado = con.sql(sql_query).df()
        return df_resultado
    except Exception as e:
        return f"Erro no SQL: {e}"
    finally:
        con.close()

# ---------------------------------------------------------
# 4. Orquestração (O Loop do Agente)
# ---------------------------------------------------------

# Pergunta do Usuário
pergunta = "Qual o total de vendas na região Norte?"

# Passo A: LLM decide o que fazer
resposta_ai = mock_llm_function_call(pergunta)

if resposta_ai:
    print(f"📩 AI Sugeriu Chamada de Função: {json.dumps(resposta_ai, indent=2)}")
    
    # Passo B: Sistema executa a ferramenta (Function Calling)
    nome_funcao = resposta_ai['function_name']
    args = resposta_ai['arguments']
    
    if nome_funcao == "executar_sql_analitico":
        resultado_real = executar_sql_analitico(args['sql_query'])
        
        print("\n📊 Resultado Final (Vindo do DuckDB):")
        print(resultado_real)
        print(f"\n💡 Explicação da IA: {args['explicacao']}")
else:
    print("AI não entendeu a pergunta.")

--- Contexto do Sistema ---

Tabela: vendas
Colunas: 
- data (TIMESTAMP)
- regiao (STRING): 'Norte', 'Sul', 'Leste', 'Oeste'
- produto (STRING)
- valor (FLOAT)


🤖 AI Recebeu: 'Qual o total de vendas na região Norte?'
🤖 AI Pensando... (Gerando SQL baseado no Schema)
📩 AI Sugeriu Chamada de Função: {
  "function_name": "executar_sql_analitico",
  "arguments": {
    "sql_query": "SELECT SUM(valor) as total FROM vendas WHERE regiao = 'Norte'",
    "explicacao": "Calculando a soma da coluna valor filtrando pela regi\u00e3o Norte."
  }
}
⚙️  Executando no DuckDB: SELECT SUM(valor) as total FROM vendas WHERE regiao = 'Norte'

📊 Resultado Final (Vindo do DuckDB):
        total
0  63728638.7

💡 Explicação da IA: Calculando a soma da coluna valor filtrando pela região Norte.


## Parte 5: Duelo de Verbosidade (Pandas vs SQL)
Visualmente, qual código é mais fácil de manter e menos propenso a erros?

In [8]:
import inspect

print("--- DUELO DE VERBOSIDADE: PANDAS vs SQL ---")

# ---------------------------------------------------------
# DESAFIO: "Calcule a média de vendas de 'Teclado' em Junho/2023"
# ---------------------------------------------------------

def abordagem_sandbox_pandas():
    # 1. Carregar (Overhead de I/O e Parsing)
    # Precisamos definir parse_dates para não falhar na lógica temporal
    df = pd.read_csv('../data/vendas_big.csv', parse_dates=['data'])
    
    # 2. Filtrar Produto (Lógica Imperativa)
    mask_produto = df['produto'] == 'Teclado'
    
    # 3. Filtrar Data (Mais lógica imperativa)
    mask_data = (df['data'].dt.year == 2023) & (df['data'].dt.month == 6)
    
    # 4. Aplicar Filtros
    df_filtrado = df[mask_produto & mask_data]
    
    # 5. Calcular e Retornar
    if len(df_filtrado) == 0:
        return 0
    return df_filtrado['valor'].mean()

def abordagem_duckdb_sql():
    # 1. Declarar a Intenção (Lógica Declarativa)
    # O motor resolve o parsing de data e leitura otimizada sozinho
    query = """
        SELECT AVG(valor) 
        FROM '../data/vendas_big.csv' 
        WHERE produto = 'Teclado' 
          AND year(data) = 2023 
          AND month(data) = 6
    """
    return duckdb.sql(query).fetchone()[0]

# ---------------------------------------------------------
# Comparativo Visual
# ---------------------------------------------------------
cod_pandas = inspect.getsource(abordagem_sandbox_pandas)
cod_duck = inspect.getsource(abordagem_duckdb_sql)

print(f"\n[Pandas] Linhas de Código: {len(cod_pandas.splitlines())}")
print(f"[DuckDB] Linhas de Código: {len(cod_duck.splitlines())}")

print("\n--- Resultado Visual ---")
print("Pandas exige que você explique 'COMO' fazer (ler, converter, mascarar, filtrar).")
print("SQL permite que você diga apenas 'O QUE' você quer.")

--- DUELO DE VERBOSIDADE: PANDAS vs SQL ---

[Pandas] Linhas de Código: 18
[DuckDB] Linhas de Código: 11

--- Resultado Visual ---
Pandas exige que você explique 'COMO' fazer (ler, converter, mascarar, filtrar).
SQL permite que você diga apenas 'O QUE' você quer.


## Parte 6: O Duelo dos Agentes (O que a IA envia)

Além da performance de execução, existe o **custo cognitivo e de tokens** para o Agente.

Abaixo, comparamos o que um Agente precisa "pensar" e enviar para a ferramenta em cada arquitetura.

### 🔴 Abordagem 1: Sandbox Python (O "Canhão")
O agente precisa atuar como um Engenheiro de Dados completo: importando bibliotecas, tratando erros de tipagem, lidando com I/O e lógica imperativa.

```json
{
  "function": "executar_codigo_python",
  "arguments": {
    "codigo": "import pandas as pd\n\n# 1. Tenta carregar CSVs gigantes para a RAM\nfluxo_caixa = pd.read_csv('/mnt/data/fluxo_caixa.csv', sep=';')\n\n# 2. Tenta corrigir tipagem manualmente (Lento e propenso a erro)\nfor col in fluxo_caixa.columns:\n    if fluxo_caixa[col].dtype == 'object':\n        fluxo_caixa[col] = fluxo_caixa[col].str.replace('.', '').str.replace(',', '.').astype(float)\n\n# 3. Lógica de Negócio Imperativa (Recriando a roda)\nfluxo_caixa['DTEMISSAO'] = pd.to_datetime(fluxo_caixa['DTEMISSAO'], dayfirst=True)\nfluxo_caixa['inadimplente'] = (fluxo_caixa['DIASATRASO'] > 0) & (~fluxo_caixa['QUITADA'].isin(['S','B']))\n\n# 4. Agregações manuais\ntotal_receber = fluxo_caixa[fluxo_caixa['ORIGEM'] == 'RECEBER']['SALDO'].sum()\n\nprint(total_receber)"
  }
}
```

**Riscos:**
*   `NameError` / `SyntaxError` (O agente erra código).
*   `OutOfMemory` (O agente trava o container).
*   Alto consumo de tokens (Prompt gigante).

---

### 🟢 Abordagem 2: SQL + DuckDB (O "Bisturi")
O agente atua como um Analista Sênior: ele apenas declara a intenção (SQL) e deixa o motor otimizado resolver o "como".

```json
{
  "function": "executar_sql_analitico",
  "arguments": {
    "sql_query": "SELECT SUM(SALDO) as total_receber FROM fluxo_caixa WHERE ORIGEM = 'RECEBER'",
    "explicacao": "Calculando total a receber diretamente da base."
  }
}
```

**Vantagens:**
*   **Zero Data Transfer:** O agente não recebe dados brutos, apenas a resposta.
*   **Robustez:** Tipagem e leitura são resolvidos pelo Schema do banco.
*   **Eficiência:** Prompt minúsculo e execução em milissegundos.